In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

C:\Users\Tedd\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.0.0)/charset_normalizer (2.0.10) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
# Function to extract Product Title
def get_title(soup):
    try:
        # Outer Tag Object
        title = soup.find("span", attrs= {'id':'productTitle'})
        
        # Inner navigatableString Object
        title_value = title.text
        
        # title as a string value
        title_string = title_value.strip()
    
    except AttributeError:
            title_string = ""
            
    return title_string

# Function to extract Product Price
def get_price(soup):
    try:
        price = soup.find("span", attrs= {'class':'a-offscreen'}).string.strip()
    
    except AttributeError:
            
            try:
                # if ther is some deal price
                price = soup.find("span", attrs= {'aria-hidden':'true'}).string.strip()
            except:
                price = ""
            
    return price


# Function to extract Product Rating
def get_rating(soup):
    try:
        rating = soup.find("i", attrs= {'class':'a-icon a-icon-star a-star-5'}).string.strip()
    
    except AttributeError:
            
            try:
                # if ther is some deal price
                rating = soup.find("span", attrs= {'class':'a-icon-alt'}).string.strip()
            except:
                rating = ""
            
    return rating


# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs= {'id':'acrCustomerReviewText'}).string.strip()
    
    except AttributeError:
        review_count = ""
        
    return review_count


# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("span", attrs= {'id':'availablility'})
        available = available.find("span").string.strip()
        
    except AttributeError:
        available = "Not Available"

    return available

In [3]:
if __name__ == "__main__":
    # headers for requests
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36',"Accept-Language":'en-US,en;q=0.5'})

    # Webpage Url
    URL = "https://www.amazon.in/s?k=xbox&ref=nb_sb_noss_2"
    
    # HTTP request
    webpage = requests.get(URL, headers=HEADERS)
    
    # Soup Object contaning all data
    soup = BeautifulSoup(webpage.content, "html.parser")
    
    # Fetch links as list of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # store the links
    links_list = []
    
    # Loop for extracting links from Tag objects
    for link in links:
        links_list.append(link.get('href'))
        
    d = {'title':[], 'price':[], 'rating':[], 'reviews':[], 'availability':[]}
    
    # loop for extracting product detials from each link
    for link in links_list:
        new_webpage = requests.get('https://www.amazon.in' + link, headers=HEADERS)
        
        new_soup = BeautifulSoup(new_webpage.content, "html.parser")
        
        # Fucntion calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))
        
        
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('',np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv('amazon_data.csv',header = True, index=False)
    

In [4]:
amazon_df

,title,price,rating,reviews,availability
1,Xbox Series X,"₹52,990.00",4.9 out of 5 stars,374 ratings,Not Available
2,Xbox Series S,"₹31,900.00",4.6 out of 5 stars,"1,541 ratings",Not Available
3,(Renewed) Xbox Series S,"₹25,500.00",3.6 out of 5 stars,22 ratings,Not Available
4,Xbox series S With Extra Controller 512 GB (Wh...,Page 1 of 1,4.2 out of 5 stars,26 ratings,Not Available
5,Grand Theft V - Premium Edition (Xbox One),"₹1,799.00",4.4 out of 5 stars,203 ratings,Not Available
6,Call Of Duty : Modern Warfare II | Cross Gen E...,"₹5,999.00",2.6 out of 5 stars,3 ratings,Not Available
7,Cricket 22 International Edition (Xbox),"₹1,899.00",2.9 out of 5 stars,16 ratings,Not Available
8,(Renewed) Microsoft Xbox One S Devil May Cry 5...,"₹36,273.00",Previous page,,Not Available
9,Microsoft Xbox Series X/S Wireless Controller ...,"₹5,390.00",4.4 out of 5 stars,400 ratings,Not Available
10,Xbox Game Pass Ultimate: 1 Month Membership (X...,"₹1,099.00",4.1 out of 5 stars,55 ratings,Not Available
